<a href="https://colab.research.google.com/github/Shahidazad/TensorFlow/blob/main/TensorFlow_NLP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
17473536/17464789 [==============================] - 0s 0us/step


In [ ]:
len(train_data[1])

189

In [ ]:
len(train_data[0])

218

In [ ]:
# len of data r different but NN work on same len data so we convt data in same len

In [ ]:
train_data = sequence.pad_sequences(train_data, MAXLEN) # if words more than 250 trim words
test_data = sequence.pad_sequences(test_data, MAXLEN) # if words less than 250 add 0s


In [ ]:
# model creation

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train model and compile it

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)




Epoch 1/10
625/625 [==============================] - 15s 13ms/step - loss: 0.4183 - acc: 0.8101 - val_loss: 0.2948 - val_acc: 0.8842
Epoch 2/10
625/625 [==============================] - 7s 12ms/step - loss: 0.2370 - acc: 0.9098 - val_loss: 0.2859 - val_acc: 0.8758
Epoch 3/10
625/625 [==============================] - 8s 12ms/step - loss: 0.1836 - acc: 0.9329 - val_loss: 0.3414 - val_acc: 0.8892
Epoch 4/10
625/625 [==============================] - 8s 12ms/step - loss: 0.1502 - acc: 0.9471 - val_loss: 0.3127 - val_acc: 0.8896
Epoch 5/10
625/625 [==============================] - 9s 14ms/step - loss: 0.1317 - acc: 0.9536 - val_loss: 0.2787 - val_acc: 0.8882
Epoch 6/10
625/625 [==============================] - 9s 14ms/step - loss: 0.1110 - acc: 0.9625 - val_loss: 0.3515 - val_acc: 0.8838
Epoch 7/10
625/625 [==============================] - 8s 14ms/step - loss: 0.0970 - acc: 0.9678 - val_loss: 0.3058 - val_acc: 0.8892
Epoch 8/10
625/625 [==============================] - 9s 15ms/step -

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)



782/782 [==============================] - 4s 6ms/step - loss: 0.5155 - acc: 0.8544
[0.5155249834060669, 0.8543999791145325]


In [ ]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


1654784/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [ ]:
# make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
#  make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])   # +ve review value is more then 0.5
                     # -ve review value is less then 0.5
positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[0.81522393]
[0.3701043]


load data set

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [ ]:
# load your own data
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

In [3]:

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))


Length of text: 1115394 characters


In [ ]:
text[:10]

'First Citi'

In [4]:
# encode each character as integer
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)


In [5]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [6]:
# convt numeric to text 

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [7]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples 
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


In [9]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry




In [10]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [13]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data 
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [14]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-2.9517596e-03 -1.3645015e-03  3.0049453e-03 ...  1.5971970e-04
    4.6030786e-03 -4.2572394e-03]
  [-1.1231718e-03 -1.4847587e-03  4.0665371e-03 ... -9.6902804e-04
    2.1882514e-03 -4.3010915e-04]
  [ 1.2395799e-04 -2.1531675e-03  4.2029154e-03 ... -2.4640304e-03
    4.8488475e-04  2.5326177e-03]
  ...
  [ 4.8102620e-03 -4.0732515e-03  5.7494442e-04 ...  1.0025585e-02
   -1.4560442e-02 -2.4102239e-03]
  [-1.1356459e-03 -2.8359997e-03  2.7968120e-03 ...  9.1074994e-03
   -7.0728296e-03 -6.6837743e-03]
  [ 7.5440328e-03  4.0478846e-03  2.8647778e-03 ...  9.8696072e-03
   -1.3563592e-02 -6.0201399e-03]]

 [[ 3.6676445e-03  1.8619094e-04 -2.8894444e-03 ...  3.5013363e-03
   -6.0239132e-04 -2.7726043e-03]
  [-1.4306849e-04 -3.3149240e-04 -1.1266513e-03 ...  1.0442856e-03
    4.8812712e-04 -4.4053313e-03]
  [ 2.5482974e-03  3.7827236e-03 -2.8645676e-03 ... -1.5194742e-03
    4.4145723e-04 -4.8154509e-03]
  ...
  [-7.1699326e-03 -2.2228612e-03 -5.4799668e-03 ... -6.2703863e

In [15]:
#  examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)


100
tf.Tensor(
[[-0.00295176 -0.0013645   0.00300495 ...  0.00015972  0.00460308
  -0.00425724]
 [-0.00112317 -0.00148476  0.00406654 ... -0.00096903  0.00218825
  -0.00043011]
 [ 0.00012396 -0.00215317  0.00420292 ... -0.00246403  0.00048488
   0.00253262]
 ...
 [ 0.00481026 -0.00407325  0.00057494 ...  0.01002559 -0.01456044
  -0.00241022]
 [-0.00113565 -0.002836    0.00279681 ...  0.0091075  -0.00707283
  -0.00668377]
 [ 0.00754403  0.00404788  0.00286478 ...  0.00986961 -0.01356359
  -0.00602014]], shape=(100, 65), dtype=float32)


In [16]:
time_pred = pred[0] # look at a prediction 
print(len(time_pred))
print(time_pred) # prediction %


65
tf.Tensor(
[-0.00295176 -0.0013645   0.00300495  0.00437595 -0.00615492 -0.00459834
 -0.00536919  0.00737626  0.00311044 -0.0051773   0.00169396  0.01053201
 -0.00149254 -0.00037544 -0.00626316  0.00407868  0.00345898  0.00192575
  0.00076716  0.00140868  0.00459421 -0.00344135 -0.00970758 -0.00272729
 -0.00128328 -0.00153946 -0.00397846  0.00217295  0.0022055  -0.00108933
  0.00029179  0.00717547  0.0030545   0.00392187 -0.00021276  0.00249266
  0.00177693  0.00220856  0.00609797 -0.00066501 -0.00272297  0.00782378
 -0.00029106  0.00055294 -0.00340767  0.00149444 -0.00017547  0.00125392
 -0.00707309 -0.00225211  0.00012698 -0.00121573  0.00159804 -0.00217163
 -0.00444165 -0.00148282 -0.00221159  0.00315557 -0.00113268 -0.00518588
 -0.00156024  0.00431837  0.00015972  0.00460308 -0.00425724], shape=(65,), dtype=float32)


In [17]:
# If we want to determine the predicted character we need to sample the output distribution 
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars # predicted charcters


"ZCW,VTDfmcFs&;ar:JgHoyjVBS\nHoE,&UTjTXVSnaWcB  pNIpvKYlxwxNyZosbh3!,DfG\nymoXVyq?kwjFVk;sIo;E'UHvmgNUv"

In [18]:
# we need to create a loss function that can compare that output to the expected
#  output and give us some numeric value representing how close the two were

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)



In [22]:
model.compile(optimizer='adam', loss=loss)  # compile model

In [42]:
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
# training data

history = model.fit(data, epochs=50, callbacks=[checkpoint_callback]) 

Epoch 1/50
172/172 [==============================] - 15s 65ms/step - loss: 2.5779
Epoch 2/50
172/172 [==============================] - 12s 66ms/step - loss: 1.8804
Epoch 3/50
172/172 [==============================] - 13s 67ms/step - loss: 1.6317
Epoch 4/50
172/172 [==============================] - 13s 68ms/step - loss: 1.4994
Epoch 5/50
172/172 [==============================] - 13s 70ms/step - loss: 1.4203
Epoch 6/50
172/172 [==============================] - 13s 71ms/step - loss: 1.3632
Epoch 7/50
172/172 [==============================] - 14s 73ms/step - loss: 1.3186
Epoch 8/50
172/172 [==============================] - 14s 74ms/step - loss: 1.2794
Epoch 9/50
172/172 [==============================] - 14s 72ms/step - loss: 1.2435
Epoch 10/50
172/172 [==============================] - 14s 72ms/step - loss: 1.2086
Epoch 11/50
172/172 [==============================] - 13s 71ms/step - loss: 1.1731
Epoch 12/50
172/172 [==============================] - 14s 73ms/step - loss: 1.1370
E

In [43]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [44]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [39]:
 tf.train.latest_checkpoint

<function tensorflow.python.training.checkpoint_management.latest_checkpoint>

In [28]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [29]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeour best exacting rung, and by earth away.

LADY ANNE:
Why, thou hast slaughter'd, and these detestable rage,
Attern of the Capitol; whom then on thither
Hath been thus cauth my poor host from twhither.
The time 'twixt him to such power.
Uncran, away; AD:
No, he is in that state may dear her lives shall follow
Ere stuff'd me about, never saw I see
you and you have made no offend it. Sir, you show'd like it in my life,
Have my disturb'd there chance hath Ros, then,
And therefore pardon me your native provenders,
And take her here or elsewhere to thy humour friends, God say;
Every gently rance, and lead the murderers!
Disprehends
That had not well for Claudio as well forswear.
Look, he rells now to keep my page, my lord.

YORK:
Ay, with my soul, I would be d life and sent for wall;
My leave f
